<a href="https://colab.research.google.com/github/faezesarlakifar/AllerTrans/blob/main/modeling/1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @markdown import necessaries
from google.colab import drive
from tqdm.notebook import tqdm
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score

In [ ]:
# @markdown mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/allergen-detection/embeddings/'

In [ ]:
# @markdown load dataset

esm_path = path+'esm-embeddings-with-id/'
esm_train = pd.read_csv(esm_path+'esm_train.csv', index_col=0)
esm_test = pd.read_csv(esm_path+'esm_test.csv', index_col=0)

protbert_path = path+'protBERT-embeddings-with-id/'
protbert_train = pd.read_csv(protbert_path+'protBERT_train.csv', index_col=0)
protbert_test = pd.read_csv(protbert_path+'protBERT_test.csv', index_col=0)

In [ ]:
esm_train.head()

,0,1,2,3,4,5,6,7,8,9,...,1272,1273,1274,1275,1276,1277,1278,1279,Label,id
0,-0.003870,-0.033864,-0.047640,0.042528,0.008789,-0.115301,0.116311,-0.056773,-0.034452,0.007469,...,0.006811,0.046086,-0.028301,0.006903,0.048489,-0.044930,-0.022138,0.003444,0,N_98814
1,-0.027616,-0.047448,-0.092680,-0.037921,-0.064493,-0.085917,0.032686,-0.104749,0.044072,0.136332,...,-0.005730,-0.007972,-0.110330,0.143354,0.079251,-0.200938,-0.064969,-0.010540,1,P_8856
2,-0.060472,-0.009816,-0.085757,-0.042129,-0.005332,-0.114610,-0.023706,-0.015580,0.004648,0.080403,...,-0.015167,-0.000249,-0.056284,0.046674,0.075886,-0.093865,-0.053575,0.069379,1,P_7716
3,-0.001879,-0.132213,0.030315,0.017521,0.093525,-0.132252,0.067915,0.093191,0.031767,0.070513,...,-0.122606,-0.032942,0.069062,-0.176981,0.018370,-0.074525,0.126718,0.107524,0,N_326869
4,-0.004742,-0.012900,-0.069494,0.027048,-0.110435,-0.009641,0.055032,-0.205043,0.036597,0.031489,...,0.036046,-0.113270,-0.020870,-0.023917,0.021013,-0.146265,-0.051615,0.176472,1,P_8031


In [ ]:
esm_test.head()

,0,1,2,3,4,5,6,7,8,9,...,1272,1273,1274,1275,1276,1277,1278,1279,Label,id
0,-0.013756,-0.056206,-0.010463,0.078954,-0.013414,-0.036181,0.101913,-0.078302,-0.029859,0.155479,...,-0.108262,-0.018523,0.023435,-0.086668,0.084220,-0.138123,0.000659,0.111366,0,N_528450
1,0.138220,-0.015468,-0.014704,-0.045518,-0.017561,0.011617,-0.093774,-0.057526,-0.016540,0.202359,...,-0.109726,0.094971,0.022741,-0.153795,0.289212,-0.096826,-0.012760,-0.088842,0,N_479266
2,0.008007,-0.054583,-0.012078,0.037711,-0.046313,-0.094285,0.004338,-0.064758,-0.096635,0.059523,...,-0.036893,0.058147,-0.039131,0.076175,0.136362,-0.062218,-0.023998,0.022915,1,P_271
3,-0.074227,-0.024113,-0.001488,-0.026625,-0.056001,0.005185,-0.002422,-0.120850,-0.008313,-0.003280,...,-0.065336,0.041332,-0.066702,0.075858,-0.057383,-0.165823,0.100414,0.105647,0,N_217429
4,0.035093,-0.027917,0.017686,0.011974,0.123146,-0.051364,-0.024959,0.176133,-0.043006,-0.027118,...,-0.204552,-0.052534,-0.056887,0.034603,0.107830,-0.013466,0.075048,-0.013248,1,P_7456


In [ ]:
protbert_train.head()

,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,Label,id
0,0.02951,-0.038240,0.042880,-0.03029,-0.01486,0.09100,-0.02155,-0.07270,0.08460,-0.00852,...,0.017780,-0.064150,0.036220,-0.07806,-0.014550,-0.009850,0.03534,0.02070,1,P_194
1,0.02860,-0.078300,0.044460,-0.03784,0.01002,0.10065,-0.03247,-0.06560,0.07630,-0.01620,...,0.009390,-0.072000,0.019970,-0.10583,-0.012480,-0.009700,0.04852,0.02205,1,P_6357
2,-0.04730,-0.002228,-0.031100,-0.01897,0.02316,0.01805,0.02126,-0.07916,-0.03537,-0.05865,...,-0.019170,0.001675,0.035900,0.00710,0.021940,-0.014656,0.03111,-0.01790,0,N_341543
3,0.01968,0.079900,0.001131,0.03240,0.02434,-0.07947,-0.01135,-0.12840,-0.03787,0.02830,...,-0.000449,-0.206700,-0.005222,0.01127,0.012085,0.014510,0.04248,-0.02217,0,N_257764
4,0.00896,0.022220,0.062230,0.00716,0.03140,0.03087,-0.04764,-0.00948,0.04837,0.02878,...,-0.037500,-0.075500,-0.021120,0.04892,-0.012870,0.023820,-0.04000,-0.06240,0,N_243343


In [ ]:
protbert_test.head()

,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,Label,id
0,-0.01852,0.08295,0.03818,-0.01380,-0.015580,-0.020190,0.04840,-0.03552,-0.000656,-0.060150,...,-0.028060,0.01277,0.047550,-0.03232,-0.02292,0.052200,0.02275,-0.023390,1,P_2249
1,-0.02400,0.00681,0.03004,-0.01816,-0.046100,-0.064500,-0.03568,-0.14220,0.004387,-0.063600,...,-0.007454,-0.04565,0.000140,-0.07220,-0.03534,0.021550,0.07184,0.006330,1,P_8311
2,-0.01677,-0.04846,0.03910,0.00860,-0.052100,-0.074800,-0.00942,-0.06480,-0.004760,-0.021060,...,0.003250,-0.04965,-0.008705,-0.03333,-0.04065,-0.021400,0.04794,0.054840,1,P_9307
3,0.01593,-0.05563,0.02385,-0.04205,-0.000392,-0.002066,-0.01698,-0.18870,0.046720,0.049530,...,0.042270,-0.12450,0.048220,0.06020,0.05258,0.003897,-0.00058,-0.027620,0,N_238995
4,-0.00402,-0.08496,0.06910,0.04944,0.025000,-0.015114,-0.06665,-0.06140,-0.004950,0.002924,...,-0.007980,-0.02780,0.069300,0.03119,-0.00836,-0.053560,0.05910,0.003437,0,N_255457


In [ ]:
# @title Concatenate training DataFrames
train = pd.merge(
    left=esm_train,
    right=protbert_train,
    on='id'
)
train.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,1015_y,1016_y,1017_y,1018_y,1019_y,1020_y,1021_y,1022_y,1023_y,Label_y
0,-0.003870,-0.033864,-0.047640,0.042528,0.008789,-0.115301,0.116311,-0.056773,-0.034452,0.007469,...,-0.058780,-0.01512,0.01888,0.02798,0.005867,-0.03363,-0.04044,-0.02450,0.034730,0
1,-0.027616,-0.047448,-0.092680,-0.037921,-0.064493,-0.085917,0.032686,-0.104749,0.044072,0.136332,...,0.034600,0.01502,-0.07150,0.05087,-0.082300,-0.01258,-0.01215,0.02357,0.011284,1
2,-0.060472,-0.009816,-0.085757,-0.042129,-0.005332,-0.114610,-0.023706,-0.015580,0.004648,0.080403,...,-0.010414,0.01288,-0.05704,0.01997,0.014015,-0.02370,0.06256,0.04718,0.013390,1
3,-0.001879,-0.132213,0.030315,0.017521,0.093525,-0.132252,0.067915,0.093191,0.031767,0.070513,...,-0.043850,-0.02454,-0.02739,0.04074,0.035600,-0.04605,-0.02040,0.04890,-0.019760,0
4,-0.004742,-0.012900,-0.069494,0.027048,-0.110435,-0.009641,0.055032,-0.205043,0.036597,0.031489,...,0.000988,-0.00882,-0.02419,0.05435,-0.088200,-0.04056,-0.06082,0.02296,0.020160,1


In [ ]:
train['Label'] = train['Label_x']
train = train.drop(columns=['Label_x', 'Label_y', 'id'])
train.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,1015_y,1016_y,1017_y,1018_y,1019_y,1020_y,1021_y,1022_y,1023_y,Label
0,-0.003870,-0.033864,-0.047640,0.042528,0.008789,-0.115301,0.116311,-0.056773,-0.034452,0.007469,...,-0.058780,-0.01512,0.01888,0.02798,0.005867,-0.03363,-0.04044,-0.02450,0.034730,0
1,-0.027616,-0.047448,-0.092680,-0.037921,-0.064493,-0.085917,0.032686,-0.104749,0.044072,0.136332,...,0.034600,0.01502,-0.07150,0.05087,-0.082300,-0.01258,-0.01215,0.02357,0.011284,1
2,-0.060472,-0.009816,-0.085757,-0.042129,-0.005332,-0.114610,-0.023706,-0.015580,0.004648,0.080403,...,-0.010414,0.01288,-0.05704,0.01997,0.014015,-0.02370,0.06256,0.04718,0.013390,1
3,-0.001879,-0.132213,0.030315,0.017521,0.093525,-0.132252,0.067915,0.093191,0.031767,0.070513,...,-0.043850,-0.02454,-0.02739,0.04074,0.035600,-0.04605,-0.02040,0.04890,-0.019760,0
4,-0.004742,-0.012900,-0.069494,0.027048,-0.110435,-0.009641,0.055032,-0.205043,0.036597,0.031489,...,0.000988,-0.00882,-0.02419,0.05435,-0.088200,-0.04056,-0.06082,0.02296,0.020160,1


In [ ]:
# @title Concatenate test DataFrames
test = pd.merge(
    left=esm_test,
    right=protbert_test,
    on='id'
)
test.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,1015_y,1016_y,1017_y,1018_y,1019_y,1020_y,1021_y,1022_y,1023_y,Label_y
0,-0.013756,-0.056206,-0.010463,0.078954,-0.013414,-0.036181,0.101913,-0.078302,-0.029859,0.155479,...,-0.002998,0.026950,-0.05620,0.06690,0.02548,-0.006428,0.04312,0.06300,0.008500,0
1,0.138220,-0.015468,-0.014704,-0.045518,-0.017561,0.011617,-0.093774,-0.057526,-0.016540,0.202359,...,0.035600,-0.014534,-0.08590,0.03397,0.03726,0.012790,-0.02563,0.09100,0.027910,0
2,0.008007,-0.054583,-0.012078,0.037711,-0.046313,-0.094285,0.004338,-0.064758,-0.096635,0.059523,...,-0.038570,-0.009390,-0.03925,0.04712,0.03078,0.004520,-0.02391,0.06647,-0.003840,1
3,-0.074227,-0.024113,-0.001488,-0.026625,-0.056001,0.005185,-0.002422,-0.120850,-0.008313,-0.003280,...,-0.049500,-0.029420,-0.08580,-0.02002,0.03270,-0.029040,-0.06335,0.01161,-0.006042,0
4,0.035093,-0.027917,0.017686,0.011974,0.123146,-0.051364,-0.024959,0.176133,-0.043006,-0.027118,...,0.029170,-0.052730,-0.06920,0.03854,0.11456,-0.035220,-0.01343,0.10626,0.055600,1


In [ ]:
test['Label'] = test['Label_x']
test = test.drop(columns=['Label_x', 'Label_y', 'id'])
test.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,1015_y,1016_y,1017_y,1018_y,1019_y,1020_y,1021_y,1022_y,1023_y,Label
0,-0.013756,-0.056206,-0.010463,0.078954,-0.013414,-0.036181,0.101913,-0.078302,-0.029859,0.155479,...,-0.002998,0.026950,-0.05620,0.06690,0.02548,-0.006428,0.04312,0.06300,0.008500,0
1,0.138220,-0.015468,-0.014704,-0.045518,-0.017561,0.011617,-0.093774,-0.057526,-0.016540,0.202359,...,0.035600,-0.014534,-0.08590,0.03397,0.03726,0.012790,-0.02563,0.09100,0.027910,0
2,0.008007,-0.054583,-0.012078,0.037711,-0.046313,-0.094285,0.004338,-0.064758,-0.096635,0.059523,...,-0.038570,-0.009390,-0.03925,0.04712,0.03078,0.004520,-0.02391,0.06647,-0.003840,1
3,-0.074227,-0.024113,-0.001488,-0.026625,-0.056001,0.005185,-0.002422,-0.120850,-0.008313,-0.003280,...,-0.049500,-0.029420,-0.08580,-0.02002,0.03270,-0.029040,-0.06335,0.01161,-0.006042,0
4,0.035093,-0.027917,0.017686,0.011974,0.123146,-0.051364,-0.024959,0.176133,-0.043006,-0.027118,...,0.029170,-0.052730,-0.06920,0.03854,0.11456,-0.035220,-0.01343,0.10626,0.055600,1


In [ ]:
df_train = train
df_test = test

## Preprocess and prepare the data

In [ ]:
df_train = train
df_test = test

In [ ]:
# Convert Label to integer
df_train['Label'] = df_train['Label'].astype(int)
df_test['Label'] = df_test['Label'].astype(int)

# Split data into train and validation sets
X_train = df_train.drop('Label', axis=1)
y_train = df_train['Label']
X_test = df_test.drop('Label', axis=1)
y_test = df_test['Label']


In [ ]:
# Create new feature names
feature_names = ['feat_'+str(i) for i in range(X_train.shape[1])]

# Rename columns
X_train.columns = feature_names
X_test.columns = feature_names

In [ ]:
# Standardize the data
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_scaled = torch.from_numpy(X_train_scaled).float()
X_test_scaled = torch.from_numpy(X_test_scaled).float()

X_train = X_train_scaled
X_test = X_test_scaled

In [ ]:
# Convert to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Create Tensors from Labels
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

# CNN

In [ ]:
from torch.utils.data import TensorDataset

## Define a 1D-CNN

In [ ]:
# ConvNet Model
class ConvNet(nn.Module):
    def __init__(self, in_channels=1):
      super().__init__()

      # self.conv1 = nn.Conv1d(in_channels, 32, 3)
      # self.conv2 = nn.Conv1d(32, 64, 3)

      self.conv1 = nn.Conv1d(in_channels, 32, 3)
      self.conv2 = nn.Conv1d(32, 64, 3)

      self.conv_dropout = nn.Dropout(p=0.2)

      self.fc1 = nn.Linear(2300 * 32, 128)
      self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = ConvNet(1)
model.to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters())
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-5)

# Batch size
batch_size = 64

In [ ]:
# Convert to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Create Tensors from Labels
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

In [ ]:
# Define TensorDataset
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64)


test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64)


## Train CNN model

In [ ]:
# Training loop
epochs = 50
for epoch in range(epochs):

  for batch in train_loader:

    inputs, labels = batch

    inputs = inputs.unsqueeze(1)
    inputs = inputs.to(device)
    labels = labels.to(device)
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")



Epoch 1, Loss: 0.1783
Epoch 2, Loss: 0.1344
Epoch 3, Loss: 0.1134
Epoch 4, Loss: 0.1514
Epoch 5, Loss: 0.0865
Epoch 6, Loss: 0.0581
Epoch 7, Loss: 0.0492
Epoch 8, Loss: 0.0171
Epoch 9, Loss: 0.0052
Epoch 10, Loss: 0.0042
Epoch 11, Loss: 0.0470
Epoch 12, Loss: 0.0149
Epoch 13, Loss: 0.0015
Epoch 14, Loss: 0.0008
Epoch 15, Loss: 0.0019
Epoch 16, Loss: 0.0164
Epoch 17, Loss: 0.0002
Epoch 18, Loss: 0.0009
Epoch 19, Loss: 0.0096
Epoch 20, Loss: 0.0051
Epoch 21, Loss: 0.0247
Epoch 22, Loss: 0.0002
Epoch 23, Loss: 0.0047
Epoch 24, Loss: 0.0001
Epoch 25, Loss: 0.0001
Epoch 26, Loss: 0.0008
Epoch 27, Loss: 0.0001
Epoch 28, Loss: 0.0001
Epoch 29, Loss: 0.0000
Epoch 30, Loss: 0.0001
Epoch 31, Loss: 0.0000
Epoch 32, Loss: 0.0001
Epoch 33, Loss: 0.0000
Epoch 34, Loss: 0.0000
Epoch 35, Loss: 0.0000
Epoch 36, Loss: 0.1263
Epoch 37, Loss: 0.0214
Epoch 38, Loss: 0.0197
Epoch 39, Loss: 0.0002
Epoch 40, Loss: 0.0002
Epoch 41, Loss: 0.0005
Epoch 42, Loss: 0.0003
Epoch 43, Loss: 0.0003
Epoch 44, Loss: 0.00

In [ ]:
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
# Training loop
epochs = 200
for epoch in range(epochs):

  for batch in train_loader:

    inputs, labels = batch

    inputs = inputs.unsqueeze(1)
    inputs = inputs.to(device)
    labels = labels.to(device)
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch+1}, Loss: {loss.item():.6f}")



Epoch 1, Loss: 0.000039
Epoch 2, Loss: 0.000018
Epoch 3, Loss: 0.000013
Epoch 4, Loss: 0.001281
Epoch 5, Loss: 0.000212
Epoch 6, Loss: 0.001542
Epoch 7, Loss: 0.000182
Epoch 8, Loss: 0.000093
Epoch 9, Loss: 0.000078
Epoch 10, Loss: 0.000065
Epoch 11, Loss: 0.000054
Epoch 12, Loss: 0.000055
Epoch 13, Loss: 0.000058
Epoch 14, Loss: 0.000041
Epoch 15, Loss: 0.000313
Epoch 16, Loss: 0.006150
Epoch 17, Loss: 0.008495
Epoch 18, Loss: 0.000055
Epoch 19, Loss: 0.000040
Epoch 20, Loss: 0.000029
Epoch 21, Loss: 0.000024
Epoch 22, Loss: 0.000020
Epoch 23, Loss: 0.000013
Epoch 24, Loss: 0.000016
Epoch 25, Loss: 0.000012
Epoch 26, Loss: 0.000010
Epoch 27, Loss: 0.000013
Epoch 28, Loss: 0.087427
Epoch 29, Loss: 0.002224
Epoch 30, Loss: 0.000102
Epoch 31, Loss: 0.000059
Epoch 32, Loss: 0.000065
Epoch 33, Loss: 0.000054
Epoch 34, Loss: 0.000035
Epoch 35, Loss: 0.000034
Epoch 36, Loss: 0.000024
Epoch 37, Loss: 0.000013
Epoch 38, Loss: 0.000010
Epoch 39, Loss: 0.000007
Epoch 40, Loss: 0.000006
Epoch 41,

In [ ]:
# Testing loop
results = []
accs = []
y_preds = []
for batch in test_loader:
  inputs, labels = batch
  inputs = inputs.unsqueeze(1)
  inputs = inputs.to(device)
  labels = labels.to(device)
  # Forward pass
  outputs = model(inputs)

  # Compute accuracy
  _, y_pred = torch.max(outputs, 1)
  # results.append(y_pred == labels)
  accuracy = accuracy_score(labels.cpu(), y_pred.cpu())
  accs.append(accuracy)
  y_preds.extend(y_pred)

# # Overall accuracy
# test_acc = torch.cat(results).mean()

print(sum(accs) / len(accs))
# Metrics

0.8647833461341526


In [ ]:
# Training loop
epochs = 400
for epoch in range(epochs):

  for batch in train_loader:

    inputs, labels = batch

    inputs = inputs.unsqueeze(1)
    inputs = inputs.to(device)
    labels = labels.to(device)
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch+1}, Loss: {loss.item():.6f}")



Epoch 1, Loss: 0.215810
Epoch 2, Loss: 0.151803
Epoch 3, Loss: 0.119426
Epoch 4, Loss: 0.093491
Epoch 5, Loss: 0.074232
Epoch 6, Loss: 0.102039
Epoch 7, Loss: 0.102242
Epoch 8, Loss: 0.042959
Epoch 9, Loss: 0.041155
Epoch 10, Loss: 0.016326
Epoch 11, Loss: 0.028974
Epoch 12, Loss: 0.018118
Epoch 13, Loss: 0.051646
Epoch 14, Loss: 0.002123
Epoch 15, Loss: 0.003755
Epoch 16, Loss: 0.000340
Epoch 17, Loss: 0.000587
Epoch 18, Loss: 0.001146
Epoch 19, Loss: 0.012408
Epoch 20, Loss: 0.001981
Epoch 21, Loss: 0.032825
Epoch 22, Loss: 0.001127
Epoch 23, Loss: 0.000441
Epoch 24, Loss: 0.025392
Epoch 25, Loss: 0.118990
Epoch 26, Loss: 0.000261
Epoch 27, Loss: 0.000273
Epoch 28, Loss: 0.000225
Epoch 29, Loss: 0.008710
Epoch 30, Loss: 0.000111
Epoch 31, Loss: 0.000077
Epoch 32, Loss: 0.001106
Epoch 33, Loss: 0.000893
Epoch 34, Loss: 0.001824
Epoch 35, Loss: 0.000338
Epoch 36, Loss: 0.000012
Epoch 37, Loss: 0.000135
Epoch 38, Loss: 0.000614
Epoch 39, Loss: 0.006040
Epoch 40, Loss: 0.091159
Epoch 41,

## Evaluation results

In [ ]:
# Testing loop
results = []
accs = []
y_preds = []
for batch in test_loader:
  inputs, labels = batch
  inputs = inputs.unsqueeze(1)
  inputs = inputs.to(device)
  labels = labels.to(device)
  # Forward pass
  outputs = model(inputs)

  # Compute accuracy
  _, y_pred = torch.max(outputs, 1)
  # results.append(y_pred == labels)
  accuracy = accuracy_score(labels.cpu(), y_pred.cpu())
  accs.append(accuracy)
  y_preds.extend(y_pred)

# # Overall accuracy
# test_acc = torch.cat(results).mean()

print(sum(accs) / len(accs))
# Metrics


0.861567140296979


In [ ]:
# Testing loop
results = []
accs = []
y_preds = []
for batch in test_loader:
  inputs, labels = batch
  inputs = inputs.unsqueeze(1)
  inputs = inputs.to(device)
  labels = labels.to(device)
  # Forward pass
  outputs = model(inputs)

  # Compute accuracy
  _, y_pred = torch.max(outputs, 1)
  # results.append(y_pred == labels)
  accuracy = accuracy_score(labels.cpu(), y_pred.cpu())
  accs.append(accuracy)
  y_preds.extend(y_pred)

# # Overall accuracy
# test_acc = torch.cat(results).mean()

print(sum(accs) / len(accs))
# Metrics


0.8610631080389145


In [ ]:
# Testing loop
results = []
accs = []
y_preds = []
for batch in test_loader:
  inputs, labels = batch
  inputs = inputs.unsqueeze(1)
  inputs = inputs.to(device)
  labels = labels.to(device)
  # Forward pass
  outputs = model(inputs)

  # Compute accuracy
  _, y_pred = torch.max(outputs, 1)
  # results.append(y_pred == labels)
  accuracy = accuracy_score(labels.cpu(), y_pred.cpu())
  accs.append(accuracy)
  y_preds.extend(y_pred)

# # Overall accuracy
# test_acc = torch.cat(results).mean()

print(sum(accs) / len(accs))
# Metrics


0.8518785202252944


In [ ]:
accuracy = accuracy_score(y_test, y_preds)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_preds.cpu(), average='weighted')

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", fscore)

In [ ]:
sum(accs) / len(accs)

0.8469022017409114